# Geodes with EODAG

In this tutorial we will see how to use [EODAG](https://github.com/CS-SI/eodag) to search and download data on the [Geodes portal](https://geodes.cnes.fr).

*Disable some warnings related to geodes SSL verification disabled*

In [1]:
import requests
requests.packages.urllib3.disable_warnings() 

## 1. Credentials configuration

As explained on [Provider registration - geodes](https://eodag.readthedocs.io/en/stable/getting_started_guide/register.html#geodes), go to https://geodes-portal.cnes.fr, then login or create an account by clicking on `Log in` in the top-right corner. 

Once logged-in, create an API key in the user settings page, and use it as `apikey` in EODAG provider auth credentials:

In [2]:
import os
from getpass import getpass

os.environ["EODAG__GEODES__AUTH__CREDENTIALS__APIKEY"] = getpass("Input apikey: ")

Input apikey:  ········


## 2. Search

Let's instantiate EODAG `EODataAccessGateway`

In [10]:
from eodag import EODataAccessGateway

dag = EODataAccessGateway()

Then look for available product types on `geodes`

In [11]:
product_types = dag.list_product_types("geodes")
[pt["ID"] for pt in product_types]

['FLATSIM_AFAR_AUXILIARYDATA_PUBLIC',
 'FLATSIM_AFAR_INTERFEROGRAM_PUBLIC',
 'FLATSIM_AFAR_TIMESERIE_PUBLIC',
 'FLATSIM_ANDES_AUXILIARYDATA_PUBLIC',
 'FLATSIM_ANDES_INTERFEROGRAM_PUBLIC',
 'FLATSIM_ANDES_TIMESERIE_PUBLIC',
 'FLATSIM_BALKANS_AUXILIARYDATA_PUBLIC',
 'FLATSIM_BALKANS_INTERFEROGRAM_PUBLIC',
 'FLATSIM_BALKANS_TIMESERIE_PUBLIC',
 'FLATSIM_CAUCASE_AUXILIARYDATA_PUBLIC',
 'FLATSIM_CAUCASE_INTERFEROGRAM_PUBLIC',
 'FLATSIM_CAUCASE_TIMESERIE_PUBLIC',
 'FLATSIM_CHILI_AUXILIARYDATA_PUBLIC',
 'FLATSIM_CHILI_INTERFEROGRAM_PUBLIC',
 'FLATSIM_CHILI_TIMESERIE_PUBLIC',
 'FLATSIM_CORSE_AUXILIARYDATA_PUBLIC',
 'FLATSIM_CORSE_INTERFEROGRAM_PUBLIC',
 'FLATSIM_CORSE_TIMESERIE_PUBLIC',
 'FLATSIM_FRANCE_AUXILIARYDATA_PUBLIC',
 'FLATSIM_FRANCE_INTERFEROGRAM_PUBLIC',
 'FLATSIM_FRANCE_TIMESERIE_PUBLIC',
 'FLATSIM_INDE_AUXILIARYDATA_PUBLIC',
 'FLATSIM_INDE_INTERFEROGRAM_PUBLIC',
 'FLATSIM_INDE_TIMESERIE_PUBLIC',
 'FLATSIM_LEVANT_AUXILIARYDATA_PUBLIC',
 'FLATSIM_LEVANT_INTERFEROGRAM_PUBLIC',
 'FLATS

Search for October-2024 Sentinel 2 L1C data over South-West of France with less that 5% of cloud coverage

In [5]:
search_bbox = [1, 43, 2, 44]
search_criteria = dict(
    provider="geodes",
    productType="S2_MSI_L1C",
    start="2024-10-01",
    end="2024-10-30",
    geom=search_bbox,
    cloudCover=5,
    count=True,
)

results = dag.search_all(**search_criteria)
results

SearchResult([EOProduct(id=S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242, provider=geodes),
              EOProduct(id=S2A_MSIL1C_20241013T104001_N0511_R008_T31TCH_20241013T124120, provider=geodes)])

View found products extents and search area on a map

In [6]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map([43.5, 1.5], zoom_start=7)

# Create a layer that represents the search area in red
folium.Rectangle(
    bounds=[search_bbox[::-1][2:4], search_bbox[::-1][0:2]], 
    color="red", 
    tooltip="Search extent"
).add_to(fmap)

# Create a layer that maps the products found
folium.GeoJson(
    data=results, tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)

fmap

## 3. Download products to `/scratch/data` directory

In [7]:
paths = dag.download_all(results, output_dir="/scratch/data")
paths

Downloaded products:   0%|          | 0/2 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

['/scratch/data/S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242',
 '/scratch/data/S2A_MSIL1C_20241013T104001_N0511_R008_T31TCH_20241013T124120']

In [8]:
!tree {paths[0]}

/scratch/data/S2B_MSIL1C_20241008T103829_N0511_R008_T31TCJ_20241008T131242
├── DATASTRIP
│   └── DS_2BPS_20241008T131242_S20241008T104519
│       ├── MTD_DS.xml
│       └── QI_DATA
│           ├── FORMAT_CORRECTNESS.xml
│           ├── GENERAL_QUALITY.xml
│           ├── GEOMETRIC_QUALITY.xml
│           ├── RADIOMETRIC_QUALITY.xml
│           └── SENSOR_QUALITY.xml
├── GRANULE
│   └── L1C_T31TCJ_A039645_20241008T104519
│       ├── AUX_DATA
│       │   ├── AUX_CAMSFO
│       │   └── AUX_ECMWFT
│       ├── IMG_DATA
│       │   ├── T31TCJ_20241008T103829_B01.jp2
│       │   ├── T31TCJ_20241008T103829_B02.jp2
│       │   ├── T31TCJ_20241008T103829_B03.jp2
│       │   ├── T31TCJ_20241008T103829_B04.jp2
│       │   ├── T31TCJ_20241008T103829_B05.jp2
│       │   ├── T31TCJ_20241008T103829_B06.jp2
│       │   ├── T31TCJ_20241008T103829_B07.jp2
│       │   ├── T31TCJ_20241008T103829_B08.jp2
│       │   ├── T31TCJ_20241008T103829_B09.jp2
│       │   ├── T31TCJ_20241008T103829_B10.jp2
│       │ 